In [0]:
# Define databricks parameters and API Credentials
dbutils.widgets.text('start', '')
dbutils.widgets.text('end', '')
dbutils.widgets.text('vendor', '')

start = dbutils.widgets.get('start')
end = dbutils.widgets.get('end')
vendor = dbutils.widgets.get('vendor')

# Access Key, Access Secret and ARN Role will be taken from AWS IAM
A_KEY = 'AKI.......2GE'
A_SECRET = 'Kz54......zb20vBx'
ARN = 'arn:aws:iam::46..47:role/ebzsellingapiRole'

MarketplaceID = 'A1PA6795UKMFR9'

# Client Key, Client Secret and Refresh Token will be taken from Vendor Central API Integration Page
C_KEY, C_SECRET_KEY, C_REFRESH_TOKEN = '', '', ''

if vendor == 'SW':
    C_KEY = 'amzn1.application-oa2-client........846'
    C_SECRET_KEY = 'cf5...................d8ceac1443a1cd93'
    C_REFRESH_TOKEN = 'Atzr.............................HdflK4'

elif vendor == 'NP':
    C_KEY = 'amzn1.application-oa2-client........846'
    C_SECRET_KEY = 'cf5...................d8ceac1443a1cd93'
    C_REFRESH_TOKEN = 'Atzr.............................HdflK4'

In [0]:
from time import sleep
import requests
import re
import json
import datetime
import gzip
from requests_aws4auth import AWS4Auth

def get_datetime():
    return datetime.datetime.utcnow().strftime("%Y%m%dT%H%M%SZ")




def get_access_token():
    global C_KEY, C_SECRET_KEY, C_REFRESH_TOKEN
    r = requests.post(f"https://api.amazon.com/auth/o2/token", data={
        "grant_type":"refresh_token",
        "refresh_token":C_REFRESH_TOKEN,
        "client_id": C_KEY,
        "client_secret": C_SECRET_KEY
    })

    return r.json()['access_token']




def assume_role():
    r = requests.get(f"https://sts.amazonaws.com?Version=2011-06-15&Action=AssumeRole&RoleSessionName=Test&RoleArn={ARN}&DurationSeconds=3600", 
        auth=AWS4Auth(A_KEY, A_SECRET, 'us-east-1', 'sts'))

    return (re.findall('<AccessKeyId>(.+)</AccessKeyId>', r.text)[0], re.findall('<SecretAccessKey>(.+)</SecretAccessKey>', r.text)[0], re.findall('<SessionToken>(.+)</SessionToken>', r.text)[0])





def g(rel_url):
    global access_token, key, secret_key, session_token

    r = requests.get(f"https://sellingpartnerapi-eu.amazon.com{rel_url}", 
        headers={'x-amz-access-token': access_token},
        auth=AWS4Auth(key, secret_key, 'eu-west-1', 'execute-api', session_token=session_token))

    return(r.json())





def gg(rel_url, params):
    global access_token, key, secret_key, session_token

    r = requests.get(f"https://sellingpartnerapi-eu.amazon.com{rel_url}", 
        params=params,
        headers={'x-amz-access-token': access_token},
        auth=AWS4Auth(key, secret_key, 'eu-west-1', 'execute-api', session_token=session_token))

    return(r.json()) 




def gd(url):
    r = requests.get(url)
    return(r.content) 




def p(rel_url, data):
    global access_token, key, secret_key, session_token

    r = requests.post(f"https://sellingpartnerapi-eu.amazon.com{rel_url}",
        data=json.dumps(data),
        headers={
            'x-amz-access-token': access_token,
            'Content-Type': 'application/json'
        },
        auth=AWS4Auth(key, secret_key, 'eu-west-1', 'execute-api', session_token=session_token))

    return (r.json())





def generate(REPORT_TYPE, start, end, type):
    c_r = p('/reports/2021-06-30/reports', {
                                            "marketplaceIds": [MarketplaceID],
                                            'reportType': REPORT_TYPE,
                                            'dataStartTime': start,
                                            'dataEndTime': end,
                                            "reportOptions": { 
                                                                "reportPeriod": "DAY" ,
                                                                "sellingProgram": "RETAIL",
                                                                "distributorView": type 
                                                             }
                                            }
           )

    print(c_r)
    return c_r['reportId']





def download(report_id):
    g_r = g(f'/reports/2021-06-30/reports/{report_id}')
    print(g_r)
    doc_id = g_r['reportDocumentId']
    d_r = g(f'/reports/2021-06-30/documents/{doc_id}')
    bt = (gd(d_r['url']))
    file = (gzip.decompress(bt).decode('utf-8'))
    return file



In [ ]:
access_token = get_access_token()
key, secret_key, session_token = assume_role()

In [ ]:
# Connect to Blob Storage and Mounting. 
# sas token can be taken from the container settings > shared access tokens.
# Permissions should be given Read, Write, Delete ..(sp=racwdli&st) while generating sas token. Give expire date range widely so that dont get auth error. 
containerName = "amazon-vendors-api"
storageAccountName = "blobsaless"
sas = "?sp=racwdli&st=20229%2F7.........................PtdFkx0%3D"
config = "fs.azure.sas." + containerName+ "." + storageAccountName + ".blob.core.windows.net"

try:
    dbutils.fs.unmount("/mnt/vendor-data-api")
except:
    pass
    
try:
    dbutils.fs.mount(
      source = f"wasbs://{containerName}@{storageAccountName}.blob.core.windows.net",
      mount_point = "/mnt/vendor-data-api",
      extra_configs = {config : sas})
except:
    pass

In [ ]:
# %fs mounts

In [0]:
# Download and Put files to the blob container
def gen_down_save(vendor, start, end, type):
    report_id = generate('GET_VENDOR_SALES_REPORT', start, end, type)
    print(report_id)

    sleep(600)

    jsonFile = download(report_id)

    dbutils.fs.put('dbfs:/mnt/vendor-data-api/' + vendor + '__' + start.replace(":", "_") + '__' + end.replace(":", "_") + '__' + type + '.json', jsonFile)

    

gen_down_save(vendor, start, end, "MANUFACTURING")
gen_down_save(vendor, start, end, "SOURCING")